# Section 2: Node and Rack Hardware Overview

In [1]:
import pyspark.sql.functions as F
from util.read_and_print_df import *
from util.handle_nan import *
from util.extract_json_attributes import *
from util.plotting import *
import builtins

In [3]:
spark = get_spark_session()

Assigning 541 GB of memory per spark driver and executor, and use 126 cores.


In [5]:
df_node_hardware_info = spark.read.parquet(path_node_hardware_info)
df_node_hardware_info = get_gpu_node_col(df_node_hardware_info, 'node')
df_node_hardware_info.show(500, False)

df_prom = spark.read.parquet(path_node_dataset)
# we select only the nodes we also have Prometheus data for
df_prom_nodes = df_prom.select('node').distinct()
print(f"Select {df_prom_nodes.count()}/{df_node_hardware_info.count()} nodes with hardware info and Prometheus data")
df_node_hardware_info = df_node_hardware_info.join(df_prom_nodes, on='node', how='inner')

+------+-----------+-------------------------+--------------------+------------------+--------------+---------------------+---------+--------------------+----------------------+-------------+--------------------------------+--------------------------+-------------------------+----------------+------------+---------+----------------------+-------------+--------+
|node  |cpu_model  |cpu_core_count_per_socket|cpu_threads_per_core|cpu_tdp_per_socket|cpu_t_case_max|cpu_per_core_temp_max|cpu_count|cpu_core_count_total|cpu_memory_bytes_total|cpu_tdp_total|node_filesystem_size_bytes_total|gpu_model                 |gpu_memory_bytes_per_card|gpu_tdp_per_card|gpu_temp_max|gpu_count|gpu_memory_bytes_total|gpu_tdp_total|gpu_node|
+------+-----------+-------------------------+--------------------+------------------+--------------+---------------------+---------+--------------------+----------------------+-------------+--------------------------------+--------------------------+---------------------

Select 338/343 nodes with hardware info and Prometheus data


In [6]:
attributes = ['cpu_count', 'cpu_core_count_total', 'cpu_tdp_total', 'cpu_memory_bytes_total', 'node_filesystem_size_bytes_total', 'gpu_count', 'gpu_tdp_total', 'gpu_memory_bytes_total']

df_pd = df_node_hardware_info.toPandas()

In [7]:
df_pd_2 = df_pd.apply(lambda x: builtins.round(x / 1024**3) if 'bytes' in x.name else x)
df_pd_2 = df_pd_2.rename(columns=lambda x: x.replace('bytes', 'giga_bytes'))

df_pd_2['node_filesystem_size_giga_bytes_total'] = df_pd_2['node_filesystem_size_giga_bytes_total'] / 1024
df_pd_2 = df_pd_2.rename(columns={'node_filesystem_size_giga_bytes_total': 'node_filesystem_size_tera_bytes_total'})

In [8]:
print("CPU-only nodes:")
display(df_pd_2[df_pd_2["gpu_node"] == 0].describe())
print("Mode values:")
df_pd_2[df_pd_2["gpu_node"] == 0].drop('node', axis=1).mode()

CPU-only nodes:


,cpu_core_count_per_socket,cpu_threads_per_core,cpu_tdp_per_socket,cpu_t_case_max,cpu_per_core_temp_max,cpu_count,cpu_core_count_total,cpu_memory_giga_bytes_total,cpu_tdp_total,node_filesystem_size_tera_bytes_total,gpu_memory_giga_bytes_per_card,gpu_tdp_per_card,gpu_temp_max,gpu_count,gpu_memory_giga_bytes_total,gpu_tdp_total,gpu_node
count,287.000000,287.0,287.000000,287.000000,287.0,287.000000,287.000000,287.000000,287.000000,287.000000,287.0,287.0,287.0,287.0,287.0,287.0,287.0
mean,13.658537,1.0,112.839721,83.783972,101.0,1.348432,16.864111,108.822300,144.233449,1.826975,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,4.125029,0.0,19.334254,4.639676,0.0,0.498799,5.479045,122.056704,37.513697,0.628778,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,8.000000,1.0,85.000000,77.000000,101.0,1.000000,16.000000,96.000000,125.000000,1.739258,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,8.000000,1.0,85.000000,77.000000,101.0,1.000000,16.000000,96.000000,125.000000,1.739258,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,16.000000,1.0,125.000000,87.000000,101.0,1.000000,16.000000,96.000000,125.000000,1.739258,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,16.000000,1.0,125.000000,87.000000,101.0,2.000000,16.000000,96.000000,170.000000,1.811523,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,26.000000,1.0,150.000000,87.000000,101.0,4.000000,52.000000,2048.000000,500.000000,11.882812,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Mode values:


,cpu_model,cpu_core_count_per_socket,cpu_threads_per_core,cpu_tdp_per_socket,cpu_t_case_max,cpu_per_core_temp_max,cpu_count,cpu_core_count_total,cpu_memory_giga_bytes_total,cpu_tdp_total,node_filesystem_size_tera_bytes_total,gpu_model,gpu_memory_giga_bytes_per_card,gpu_tdp_per_card,gpu_temp_max,gpu_count,gpu_memory_giga_bytes_total,gpu_tdp_total,gpu_node
0,gold_6130,16,1,125.0,87.0,101.0,1,16,96.0,125.0,1.739258,n/a,0.0,0.0,0.0,0,0.0,0.0,0


In [9]:
print("GPU nodes:")
display(df_pd_2[df_pd_2["gpu_node"] > 0].describe())
print("Mode values:")
df_pd_2[df_pd_2["gpu_node"] > 0].drop('node', axis=1).mode()

GPU nodes:


,cpu_core_count_per_socket,cpu_threads_per_core,cpu_tdp_per_socket,cpu_t_case_max,cpu_per_core_temp_max,cpu_count,cpu_core_count_total,cpu_memory_giga_bytes_total,cpu_tdp_total,node_filesystem_size_tera_bytes_total,gpu_memory_giga_bytes_per_card,gpu_tdp_per_card,gpu_temp_max,gpu_count,gpu_memory_giga_bytes_total,gpu_tdp_total,gpu_node
count,51.000000,51.0,51.000000,51.000000,51.0,51.0,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.0
mean,10.117647,1.0,98.333333,80.176471,101.0,2.0,20.235294,296.156863,196.666667,2.348020,18.901961,268.235294,89.784314,3.882353,72.784314,1040.000000,1.0
std,3.824380,0.0,11.775681,2.251274,0.0,0.0,7.648760,314.567854,23.551362,0.207320,6.410163,14.792685,0.986179,0.475271,25.886146,134.342845,0.0
min,6.000000,1.0,85.000000,78.000000,101.0,2.0,12.000000,192.000000,170.000000,0.912109,11.000000,250.000000,89.000000,2.000000,44.000000,560.000000,1.0
25%,6.000000,1.0,85.000000,78.000000,101.0,2.0,12.000000,192.000000,170.000000,2.366211,11.000000,250.000000,89.000000,4.000000,44.000000,1000.000000,1.0
50%,12.000000,1.0,105.000000,81.000000,101.0,2.0,24.000000,192.000000,210.000000,2.366211,24.000000,280.000000,89.000000,4.000000,96.000000,1120.000000,1.0
75%,12.000000,1.0,105.000000,81.000000,101.0,2.0,24.000000,256.000000,210.000000,2.373047,24.000000,280.000000,91.000000,4.000000,96.000000,1120.000000,1.0
max,20.000000,1.0,125.000000,87.000000,101.0,2.0,40.000000,1536.000000,250.000000,2.496094,24.000000,280.000000,91.000000,4.000000,96.000000,1120.000000,1.0


Mode values:


,cpu_model,cpu_core_count_per_socket,cpu_threads_per_core,cpu_tdp_per_socket,cpu_t_case_max,cpu_per_core_temp_max,cpu_count,cpu_core_count_total,cpu_memory_giga_bytes_total,cpu_tdp_total,node_filesystem_size_tera_bytes_total,gpu_model,gpu_memory_giga_bytes_per_card,gpu_tdp_per_card,gpu_temp_max,gpu_count,gpu_memory_giga_bytes_total,gpu_tdp_total,gpu_node
0,gold_5118,12,1,105.0,81.0,101.0,2,24,192.0,210.0,2.366211,NVIDIA_TITAN_RTX,24.0,280.0,89.0,4,96.0,1120.0,1


In [10]:
print("Total values of cluster:")
df_pd_2.drop(['node', 'cpu_model', 'gpu_model'], axis=1).sum()

Total values of cluster:


cpu_core_count_per_socket                 4436.00000
cpu_threads_per_core                       338.00000
cpu_tdp_per_socket                       37400.00000
cpu_t_case_max                           28135.00000
cpu_per_core_temp_max                    34138.00000
cpu_count                                  489.00000
cpu_core_count_total                      5872.00000
cpu_memory_giga_bytes_total              46336.00000
cpu_tdp_total                            51425.00000
node_filesystem_size_tera_bytes_total      644.09082
gpu_memory_giga_bytes_per_card             964.00000
gpu_tdp_per_card                         13680.00000
gpu_temp_max                              4579.00000
gpu_count                                  198.00000
gpu_memory_giga_bytes_total               3712.00000
gpu_tdp_total                            53040.00000
gpu_node                                    51.00000
dtype: float64

Generate tables and boxplots about hardware info on rack level.

In [11]:
df_node_hardware_info_rack = df_node_hardware_info.withColumn('rack', F.substring('node', 1, 3))
df_node_hardware_info_rack = df_node_hardware_info_rack.groupBy('rack').agg(
    F.count('node').alias('node_count_sum'),
    *[F.sum(a).alias(f'{a}_sum') for a in attributes],
    F.expr(f'percentile_approx(gpu_node, 0.5)').alias('gpu_rack')
    )

df_pd_rack = df_node_hardware_info_rack.toPandas()

In [12]:
df_pd_rack_2 = df_pd_rack.apply(lambda x: builtins.round(x / 1024**3) if 'bytes' in x.name else x)
df_pd_rack_2 = df_pd_rack_2.rename(columns=lambda x: x.replace('bytes', 'giga_bytes'))

df_pd_rack_2['node_filesystem_size_giga_bytes_total_sum'] = df_pd_rack_2['node_filesystem_size_giga_bytes_total_sum'] / 1024
df_pd_rack_2 = df_pd_rack_2.rename(columns={'node_filesystem_size_giga_bytes_total_sum': 'node_filesystem_size_tera_bytes_total_sum'})

In [13]:
print("CPU-only racks:")
display(df_pd_rack[df_pd_rack["gpu_rack"] == 0].describe())
print("Mode values:")
df_pd_rack_2[df_pd_rack_2["gpu_rack"] == 0].drop('rack', axis=1).mode()

CPU-only racks:


,node_count_sum,cpu_count_sum,cpu_core_count_total_sum,cpu_tdp_total_sum,cpu_memory_bytes_total_sum,node_filesystem_size_bytes_total_sum,gpu_count_sum,gpu_tdp_total_sum,gpu_memory_bytes_total_sum,gpu_rack
count,11.000000,11.000000,11.000000,11.000000,1.100000e+01,1.100000e+01,11.0,11.0,11.0,11.0
mean,26.090909,35.181818,440.000000,3763.181818,3.048646e+12,5.241332e+13,0.0,0.0,0.0,0.0
std,11.255706,18.829860,142.076036,1434.716475,3.675339e+11,1.767697e+13,0.0,0.0,0.0,0.0
min,1.000000,4.000000,48.000000,500.000000,2.199023e+12,1.306563e+13,0.0,0.0,0.0,0.0
25%,30.000000,30.500000,480.000000,3812.500000,3.092376e+12,5.832357e+13,0.0,0.0,0.0,0.0
50%,31.000000,32.000000,496.000000,4000.000000,3.195456e+12,5.976675e+13,0.0,0.0,0.0,0.0
75%,32.000000,46.000000,512.000000,4550.000000,3.298535e+12,6.119195e+13,0.0,0.0,0.0,0.0
max,32.000000,62.000000,512.000000,5270.000000,3.298535e+12,6.175897e+13,0.0,0.0,0.0,0.0


Mode values:


,node_count_sum,cpu_count_sum,cpu_core_count_total_sum,cpu_tdp_total_sum,cpu_memory_giga_bytes_total_sum,node_filesystem_size_tera_bytes_total_sum,gpu_count_sum,gpu_tdp_total_sum,gpu_memory_giga_bytes_total_sum,gpu_rack
0,32,32,512,4000.0,3072.0,55.65332,0,0.0,0.0,0


In [14]:
print("GPU racks:")
display(df_pd_rack[df_pd_rack["gpu_rack"] == 1].describe())
print("Mode values:")
df_pd_rack_2[df_pd_rack_2["gpu_rack"] == 1].drop('rack', axis=1).mode()

GPU racks:


,node_count_sum,cpu_count_sum,cpu_core_count_total_sum,cpu_tdp_total_sum,cpu_memory_bytes_total_sum,node_filesystem_size_bytes_total_sum,gpu_count_sum,gpu_tdp_total_sum,gpu_memory_bytes_total_sum,gpu_rack
count,10.000000,10.000000,10.000000,10.000000,1.000000e+01,1.000000e+01,10.000000,10.000000,1.000000e+01,10.0
mean,5.100000,10.200000,103.200000,1003.000000,1.621780e+12,1.316764e+13,19.800000,5304.000000,3.985730e+11,1.0
std,1.286684,2.573368,38.031566,239.678025,1.295058e+12,3.636722e+12,5.996295,1558.711719,1.666364e+11,0.0
min,3.000000,6.000000,48.000000,550.000000,7.559142e+11,6.220811e+12,10.000000,2800.000000,1.975685e+11,1.0
25%,4.250000,8.500000,75.000000,877.500000,1.030792e+12,1.137899e+13,17.000000,4610.000000,2.641405e+11,1.0
50%,5.000000,10.000000,108.000000,1035.000000,1.202591e+12,1.301101e+13,20.000000,5600.000000,3.715147e+11,1.0
75%,5.750000,11.500000,120.000000,1050.000000,1.597728e+12,1.500218e+13,23.000000,5900.000000,5.153961e+11,1.0
max,7.000000,14.000000,168.000000,1470.000000,5.153961e+12,1.826352e+13,28.000000,7840.000000,7.215545e+11,1.0


Mode values:


,node_count_sum,cpu_count_sum,cpu_core_count_total_sum,cpu_tdp_total_sum,cpu_memory_giga_bytes_total_sum,node_filesystem_size_tera_bytes_total_sum,gpu_count_sum,gpu_tdp_total_sum,gpu_memory_giga_bytes_total_sum,gpu_rack
0,5,10,120,1050.0,960.0,11.833008,20,5600.0,480.0,1


In [16]:
df_slurm = spark.read.parquet(path_job_dataset)
print("Job Dataset Size:", df_slurm.count(), "x", len(df_slurm.columns), "Start Date:", df_slurm.select(F.min("start_date")).first()[0], "End Date:", df_slurm.select(F.max("end_date")).first()[0])
df_slurm.show(5, False)

df_prom = spark.read.parquet(path_node_dataset)
print("Node Dataset Size:", df_prom.count(), "x", len(df_prom.columns), "Start Date:", df_prom.select(F.min("timestamp")).first()[0], "End Date:", df_prom.select(F.max("timestamp")).first()[0])
df_prom.show(5, False)

df_combined = spark.read.parquet(path_job_node_joined_dataset)
print("Job-Node Dataset Size:", df_combined.count(), "x", len(df_combined.columns), "Start Date:", df_combined.select(F.min("timestamp")).first()[0], "End Date:", df_combined.select(F.max("timestamp")).first()[0])
df_combined.show(5, False)

Job Dataset Size: 1596963 x 9 Start Date: 2021-12-26 23:06:31 End Date: 2022-11-01 13:59:18
+-----+-------------------+-------------------+-------------------+------+---------+--------+--------+---------+
|id   |submit_date        |start_date         |end_date           |node  |nodetypes|numnodes|numcores|state    |
+-----+-------------------+-------------------+-------------------+------+---------+--------+--------+---------+
|58607|2022-01-14 17:39:50|2022-01-15 06:39:07|2022-01-15 14:30:47|r15n11|normal   |1       |16      |COMPLETED|
|58786|2022-01-14 17:39:50|2022-01-15 06:39:07|2022-01-15 16:27:53|r11n8 |normal   |1       |16      |COMPLETED|
|58669|2022-01-14 17:39:50|2022-01-15 06:41:37|2022-01-15 15:41:45|r14n23|normal   |1       |16      |COMPLETED|
|58548|2022-01-14 17:39:50|2022-01-15 06:42:37|2022-01-15 13:38:40|r12n3 |normal   |1       |16      |COMPLETED|
|58848|2022-01-14 17:39:50|2022-01-15 06:46:38|2022-01-15 17:11:13|r25n29|normal   |1       |16      |COMPLETED|
+---